In [228]:
import sys
import os

# Add the path to the parent directory of `src` to the sys.path
sys.path.append(os.path.abspath('C:\\Away\\FMP\\New\\guiV2_1-main_2\\src'))

%load_ext autoreload
%autoreload 2

# Importing Masha's functions
    
from trash.simulation_tools import compute_flows, run_simulation
import trash.nernst_potentials as P

import trash.ionic_fluxes as i_flux
import trash.dep_functions as dep_funct

# Importing the backend package

# from backend.backend.snapshotable import Snapshotable
from backend.backend.vesicle import VesicleConfig, Vesicle
from backend.backend.exterior import ExteriorConfig, Exterior
from backend.backend.ion_channels import (IonChannelConfig, 
                                          IonChannel, 
                                          ASORChannelConfig, 
                                          ASORChannel, 
                                          CLCChannelConfig, 
                                          CLCChannel, 
                                          TPCChannelConfig, 
                                          TPCChannel, 
                                          NHEChannelConfig, 
                                          NHEChannel, 
                                          VATPaseChannelConfig, 
                                          VATPaseChannel,
                                          HLeakChannelConfig,
                                          HLeakChannel,
                                          KChannelConfig,
                                          KChannel
                                        )

import numpy as np
from math import exp, log

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [229]:
# mapping of chemicals to numerical indices used to index the state vectors
Cl_idx = 0
Na_idx = 1
H_idx = 2
K_idx = 3

number_of_ions = 4
T = 1000 # total time (in seconds)
dt = 0.001 # duration of integration window (in seconds). Has to be 0.001 or less to make integration stable

RT = 2578.5871 # gas constant x temperature in Kelvin
F = 96485.0 # Faraday constant
c_spec = 0.01  # membrane capacitance per surface area F/(m**2)
buffer_capacity_t0=5.0*1e-4 # initial buffer capacity
U0 = 40*1e-3 # membrane potential in V 

r=1.3e-6 # radius of the vesicle in m
V0 = (4.0/3.0) * np.pi * (r**3) # Initial volume of the vesicle in m3
A0 = 4.0 * np.pi * (r**2) # initial surface area of the vesicle in m2
A_from_V_const=(36.0*np.pi)**(1/3) # constant necessary for calculation surface area from the volume

C0 = A0 * c_spec # inititial membrane capacitance

Cl_o_concentration = 20*1e-3 # external Cl concentraion in M
Cl_i_concentration = 159*1e-3

Na_o_concentration=10*1e-3 # external Na concentraion in M
Na_i_concentration=150*1e-3 # internal Na concentraion in M

K_o_concentration=140*1e-3 # external Cl concentraion in M
K_i_concentration=5*1e-3 # internal K concentraion in M

pH_o =  7.2 # external pH
pH_i =  7.4 # internal pH

hfree_o_concentration = 10**(-pH_o) # concentration of free external protons in M
hfree_i_concentration = 10**(-pH_i) # concentration of free internal protons in M

htotal_o_concentration = hfree_o_concentration/ buffer_capacity_t0 # concentration of total external protons in M
htotal_i_concentration = hfree_i_concentration/ buffer_capacity_t0 # concentration of total intenral protons in M

htotal_i_amount = htotal_i_concentration*V0*1000 # amount of total internal protons in moles

def initialize_internal_concentrations(Cl_i_concentration = 159*1e-3):
   
    # Cl_i_concentration= 159*1e-3 # internal Cl concentraion in M
    # Cl_i_concentration=1*1e-3 # absent internal Cl concentration in M
    # Cl_i_concentration= 9*1e-3 # Cl replacement condition (internal Cl concentration in M)

    Q0=U0*C0 # initial total charge
    X_amount=(Q0/F)-((Na_i_concentration+K_i_concentration+htotal_i_concentration-Cl_i_concentration)*V0*1000) # initial amount of unaccouted ions in moles
    X_concentration=X_amount/(V0*1000) # initial concentration of unaccounted ions in moles

    internal_ions_amounts=[Cl_i_concentration*V0*1000,  Na_i_concentration*V0*1000, htotal_i_concentration*V0*1000, K_i_concentration*V0*1000] # vector of amounts of ions in moles
    external_ions_concentrations = [Cl_o_concentration, Na_o_concentration, htotal_o_concentration, K_o_concentration] # vector of concentrations of external ions
    internal_ions_concentrations = [Cl_i_concentration, Na_i_concentration, htotal_i_concentration, K_i_concentration] # vector of concentrations of internal ions

    Sum_initial_amounts = internal_ions_amounts[Cl_idx] + internal_ions_amounts[Na_idx] + abs(X_amount) + internal_ions_amounts[K_idx] # sum of amounts of all ions

    return X_amount, external_ions_concentrations, internal_ions_amounts, internal_ions_concentrations, Sum_initial_amounts

conductance_ASOR = 8e-5
conductance_CLC = 1e-7
conductance_TPC = 2e-6
conductance_NHE = 1
conductance_VATPase = 8e-9
conductance_Hleak = 1.6e-8
conductance_K = 1

U_ASOR0 = dep_funct.v_dependence_ASOR(U0, 80.0, -40*1e-3)
pH_k2_ASOR = 3.0
pH_half_ASOR = 5.4
U_k2_ASOR = 80.0
U_half_ASOR = -40*1e-3

pH_k2_CLC = 1.5
pH_half_CLC = 5.5
U_k2_CLC = 80.0
U_half_CLC = -40*1e-3

U_ATP = 0.27
t0 = 1

In [230]:
NP_ASOR_M = P.nernst_potential_Cl_asor(U0, Cl_o_concentration, Cl_i_concentration, RT/F)

Flux_Cl_ASOR_M = i_flux.J_cl_asor(NP_ASOR_M, 
                                  conductance_ASOR, 
                                  U0, 
                                  pH_i, 
                                  A0, 
                                  pH_k2_ASOR, 
                                  pH_half_ASOR, 
                                  U_k2_ASOR, 
                                  U_half_ASOR)



NP_CLC_M = P.nernst_potential_CLC(U0, Cl_i_concentration, Cl_o_concentration, hfree_i_concentration, hfree_o_concentration, RT, F)

U_CLC_M = dep_funct.v_dependence_ClC(U0, U_k2_CLC, U_half_CLC)
ph_CLC_M = dep_funct.pH_dependence_ClC(pH_i, pH_k2_CLC, pH_half_CLC)

Flux_Cl_CLC_M = i_flux.J_Cl_CLC(NP_CLC_M, 
                                conductance_CLC,
                                U0,
                                pH_i,
                                A0,
                                pH_k2_CLC,
                                pH_half_CLC,
                                U_k2_CLC,
                                U_half_CLC)



NP_NaTPC_M = P.nernst_potential_Na_K(U0, Na_o_concentration, Na_i_concentration, RT/F)

Flux_NaTPC_M = i_flux.J_na_tpc(NP_NaTPC_M,
                              conductance_TPC,
                              A0)


NP_NaNHE_M = P.potential_NHE(Na_o_concentration, Na_i_concentration, hfree_i_concentration, hfree_o_concentration)

Flux_NaNHE_M = i_flux.J_Na_NHE(NP_NaNHE_M,
                              conductance_NHE,
                              A0)

NP_H_CLC_M = P.nernst_potential_CLC(U0, htotal_i_concentration, htotal_o_concentration, hfree_i_concentration, hfree_o_concentration, RT, F)

Flux_H_CLC_M = i_flux.J_H_CLC(NP_H_CLC_M, 
                              conductance_CLC,
                              U0,
                              pH_i,
                              A0,
                              pH_k2_CLC,
                              pH_half_CLC,
                              U_k2_CLC,
                              U_half_CLC)

NP_H_NHE_M = P.potential_NHE(htotal_o_concentration, htotal_i_concentration, hfree_i_concentration, hfree_o_concentration)

Flux_H_NHE_M = i_flux.J_H_NHE(NP_H_NHE_M,
                              conductance_NHE,
                              A0)

NP_H_VATPase_M = P.nernst_potential_VATPase(U0, htotal_o_concentration, htotal_i_concentration, RT/F, U_ATP)

Flux_H_VATPase_M = i_flux.J_VATPase(NP_H_VATPase_M,
                                   conductance_VATPase,
                                   t0,
                                   A0)

NP_HLeak_M = P.nernst_potential_H_leak(U0, hfree_o_concentration, hfree_i_concentration, RT/F)

Flux_Hleak_M = i_flux.J_H_leak(NP_HLeak_M,
                              conductance_Hleak,
                              A0)

NP_K_M = P.nernst_potential_Na_K(U0, K_o_concentration, K_i_concentration, RT/F)

Flux_K_M = i_flux.J_k(NP_K_M,
                     conductance_K,
                     A0)

In [231]:
ASOR_config = ASORChannelConfig()

ASOR_channel = ASORChannel()

ASOR_channel.VOLTAGE_MULTIPLIER = 1
ASOR_channel.FLUX_MULTIPLIER = 1
ASOR_channel.NERNST_MULTIPLIER = 1
ASOR_channel.VOLTAGE_SHIFT = 0
ASOR_channel.nernst_constant = RT/F
ASOR_channel.nernst_potential = ASOR_channel.compute_nernst_potential(voltage=U0, 
                                                exterior_conc=Cl_o_concentration, 
                                                vesicle_conc=Cl_i_concentration)
NP_ASOR = ASOR_channel.nernst_potential


ASOR_channel.pH_dependence = ASOR_channel.compute_pH_dependence(pH_i)
ASOR_channel.voltage_dependence = ASOR_channel.compute_voltage_dependence(U0)

cond_ASOR = ASOR_channel.conductance

ASOR_channel.flux = ASOR_channel.compute_flux(voltage=U0, 
                                              exterior_conc=Cl_o_concentration, 
                                              vesicle_conc=Cl_i_concentration,
                                              area = A0)
FLUX_ASOR = ASOR_channel.flux

In [232]:
Ves_config = VesicleConfig()
Ext_config = ExteriorConfig()
Ves = Vesicle(init_buffer_capacity=5e-4, config=Ves_config)
Ext = Exterior(init_buffer_capacity=5e-4, config=Ext_config)

CLC_config = CLCChannelConfig()

CLC_channel = CLCChannel()

CLC_channel.VOLTAGE_MULTIPLIER = 1
CLC_channel.FLUX_MULTIPLIER = 2
CLC_channel.NERNST_MULTIPLIER = 1/3
CLC_channel.VOLTAGE_SHIFT = 0
CLC_channel.nernst_constant = RT/F

cond_CLC = CLC_channel.conductance

CLC_channel.pH_dependence = CLC_channel.compute_pH_dependence(pH_i)
CLC_channel.voltage_dependence = CLC_channel.compute_voltage_dependence(U0)

NP_CLC = CLC_channel.compute_nernst_potential(voltage=U0,
                                     exterior_conc=Cl_o_concentration,
                                     vesicle_conc=Cl_i_concentration,
                                     vesicle = Ves,
                                     exterior = Ext)

CLC_channel.flux = CLC_channel.compute_flux(voltage=U0, 
                                            exterior_conc=Cl_o_concentration, 
                                            vesicle_conc=Cl_i_concentration,
                                            area = A0,
                                            vesicle = Ves,
                                            exterior = Ext)
FLUX_CLC = CLC_channel.flux

In [233]:
TPC_config = TPCChannelConfig()

TPC_channel = TPCChannel()

TPC_channel.VOLTAGE_MULTIPLIER = -1
TPC_channel.FLUX_MULTIPLIER = 1
TPC_channel.NERNST_MULTIPLIER = 1
TPC_channel.VOLTAGE_SHIFT = 0
TPC_channel.nernst_constant = RT/F

cond_TPC = CLC_channel.conductance

TPC_channel.flux = TPC_channel.compute_flux(voltage=U0, 
                                            exterior_conc=Na_o_concentration, 
                                            vesicle_conc=Na_i_concentration,
                                            area = A0
                                           )
FLUX_Na_TPC = TPC_channel.flux

In [234]:
Ves_config_NHE = VesicleConfig()
Ext_config_NHE = ExteriorConfig()
Ves_NHE = Vesicle(init_buffer_capacity=5e-4, config=Ves_config)
Ext_NHE = Exterior(init_buffer_capacity=5e-4, config=Ext_config)

NHE_config = NHEChannelConfig()

NHE_channel = NHEChannel()

NHE_channel.conductance = conductance_NHE

NHE_channel.VOLTAGE_MULTIPLIER = 0
NHE_channel.FLUX_MULTIPLIER = 1
NHE_channel.NERNST_MULTIPLIER = 1
NHE_channel.VOLTAGE_SHIFT = 0
NHE_channel.nernst_constant = 1

NHE_channel.flux = NHE_channel.compute_flux(voltage=U0, 
                                            exterior_conc=Na_o_concentration, 
                                            vesicle_conc=Na_i_concentration,
                                            area = A0, 
                                            vesicle = Ves_NHE,
                                            exterior = Ext_NHE
                                            )
FLUX_Na_NHE = NHE_channel.flux

In [235]:
Ves_config2 = VesicleConfig()
Ext_config2 = ExteriorConfig()
Ves2 = Vesicle(init_buffer_capacity=5e-4, config=Ves_config2)
Ext2 = Exterior(init_buffer_capacity=5e-4, config=Ext_config2)

CLC_config2 = CLCChannelConfig()

CLC_channel2 = CLCChannel()

CLC_channel2.VOLTAGE_MULTIPLIER = 1
CLC_channel2.FLUX_MULTIPLIER = -1
CLC_channel2.NERNST_MULTIPLIER = 1/3
CLC_channel2.VOLTAGE_SHIFT = 0
CLC_channel2.nernst_constant = RT/F

cond_CLC = CLC_channel.conductance

CLC_channel2.pH_dependence = CLC_channel2.compute_pH_dependence(pH_i)
CLC_channel2.voltage_dependence = CLC_channel2.compute_voltage_dependence(U0)

CLC_channel2.flux = CLC_channel2.compute_flux(voltage=U0, 
                                             exterior_conc=htotal_o_concentration, 
                                             vesicle_conc=htotal_i_concentration,
                                             area = A0,
                                             vesicle = Ves2,
                                             exterior = Ext2)

FLUX_H_CLC = CLC_channel2.flux

In [236]:
Ves_config_NHE2 = VesicleConfig()
Ext_config_NHE2 = ExteriorConfig()
Ves_NHE2 = Vesicle(init_buffer_capacity=5e-4, config=Ves_config_NHE2)
Ext_NHE2 = Exterior(init_buffer_capacity=5e-4, config=Ext_config_NHE2)

NHE_config2 = NHEChannelConfig()

NHE_channel2 = NHEChannel()

NHE_channel2.conductance = conductance_NHE

NHE_channel2.VOLTAGE_MULTIPLIER = 0
NHE_channel2.FLUX_MULTIPLIER = -1
NHE_channel2.NERNST_MULTIPLIER = 1
NHE_channel2.VOLTAGE_SHIFT = 0
NHE_channel2.nernst_constant = 1

NHE_channel2.flux = NHE_channel2.compute_flux(voltage=U0, 
                                            exterior_conc=htotal_o_concentration, 
                                            vesicle_conc=htotal_i_concentration,
                                            area = A0, 
                                            vesicle = Ves_NHE2,
                                            exterior = Ext_NHE2
                                            )
FLUX_H_NHE = NHE_channel2.flux

In [237]:
VATPase_config = VATPaseChannelConfig()
VATPase_channel = VATPaseChannel()

VATPase_channel.time_exponent = 0
VATPase_channel.half_act_time = 0

VATPase_channel.conductance = conductance_VATPase

VATPase_channel.VOLTAGE_MULTIPLIER = 1
VATPase_channel.FLUX_MULTIPLIER = -1
VATPase_channel.NERNST_MULTIPLIER = -1
VATPase_channel.VOLTAGE_SHIFT = U_ATP
VATPase_channel.nernst_constant = RT/F

VATPase_channel.time_dependence = VATPase_channel.compute_time_dependence(t0)

FLUX_H_VATPase = VATPase_channel.compute_flux(voltage=U0,
                                    exterior_conc=htotal_o_concentration,
                                    vesicle_conc=htotal_i_concentration,
                                    area=A0,
                                    t=t0)

In [238]:
Ves_config_Hleak = VesicleConfig()
Ext_config_Hleak = ExteriorConfig()

Ves_Hleak = Vesicle(init_buffer_capacity=5e-4, config=Ves_config_Hleak)
Ext_Hleak = Exterior(init_buffer_capacity=5e-4, config=Ext_config_Hleak)

HLeak_config = HLeakChannelConfig()

HLeak_channel = HLeakChannel()

HLeak_channel.conductance = conductance_Hleak

HLeak_channel.VOLTAGE_MULTIPLIER = -1
HLeak_channel.FLUX_MULTIPLIER = 1
HLeak_channel.NERNST_MULTIPLIER = 1
HLeak_channel.VOLTAGE_SHIFT = 0
HLeak_channel.nernst_constant = RT/F

FLUX_HLeak = HLeak_channel.compute_flux(voltage=U0,
                                       exterior_conc=htotal_o_concentration, 
                                       vesicle_conc=htotal_i_concentration,
                                       area = A0,
                                       vesicle = Ves_Hleak,
                                       exterior = Ext_Hleak
                                       )

In [240]:
K_config = KChannelConfig()

K_channel = KChannel()

K_channel.conductance = conductance_K

K_channel.VOLTAGE_MULTIPLIER = -1
K_channel.FLUX_MULTIPLIER = 1
K_channel.NERNST_MULTIPLIER = 1
K_channel.VOLTAGE_SHIFT = 0
K_channel.nernst_constant = RT/F

FLUX_K = K_channel.compute_flux(voltage=U0, 
                                exterior_conc=K_o_concentration, 
                                vesicle_conc=K_i_concentration,
                                area=A0
                               )

In [241]:
print(f'Flux_Cl_ASOR_M: {Flux_Cl_ASOR_M}')
print(f'Flux_Cl_ASOR  : {FLUX_ASOR}')

print(f'Flux_Cl_CLC_M: {Flux_Cl_CLC_M}')
print(f'Flux_Cl_CLC  : {FLUX_CLC}')

print(f'Flux_NaTPC_M: {Flux_NaTPC_M}')
print(f'Flux_NaTPC  : {FLUX_Na_TPC}')

print(f'Flux_NaNHE_M: {Flux_NaNHE_M}')
print(f'Flux_NaNHE  : {FLUX_Na_NHE}')

print(f'Flux_H_CLC_M: {Flux_H_CLC_M}')
print(f'Flux_H_CLC  : {FLUX_H_CLC}')

print(f'Flux_H_NHE_M: {Flux_H_NHE_M}')
print(f'Flux_H_NHE  : {FLUX_H_NHE}')

print(f'Flux_H_VATPase_M: {Flux_H_VATPase_M}')
print(f'Flux_H_VATPase  : {FLUX_H_VATPase}')

print(f'Flux_Hleak_M: {Flux_Hleak_M}')
print(f'Flux_Hleak  : {FLUX_HLeak}')

print(f'Flux_K_M: {Flux_K_M}')
print(f'Flux_K  : {FLUX_K}')

Flux_Cl_ASOR_M: -1.0735705375934016e-22
Flux_Cl_ASOR  : -1.0735705375934016e-22
Flux_Cl_CLC_M: -6.925849492281038e-24
Flux_Cl_CLC  : -6.925849492281038e-24
Flux_NaTPC_M: -4.7729833335998434e-18
Flux_NaTPC  : -4.7729833335998434e-18
Flux_NaNHE_M: -6.729138909877076e-11
Flux_NaNHE  : -6.729138909877076e-11
Flux_H_CLC_M: -1.4686967282029442e-22
Flux_H_CLC  : -1.4686967282029442e-22
Flux_H_NHE_M: -4.7155982093076695e-27
Flux_H_NHE  : -4.7155982093076695e-27
Flux_H_VATPase_M: 2.0583693540751714e-20
Flux_H_VATPase  : 2.0583693540751714e-20
Flux_Hleak_M: -9.4097844183066e-21
Flux_Hleak  : -9.4097844183066e-21
Flux_K_M: 1.0417689204217915e-12
Flux_K  : 1.0417689204217915e-12
